In [1]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# 相交
def intersect_shapefiles(gdf1, gdf2, gdf1_path=None, gdf2_path=None, keep_columns1=None, keep_columns2=None):
    """
    对两个矢量数据进行空间相交操作
    
    参数:
    gdf1: GeoDataFrame对象或None，如为None则从gdf1_path读取
    gdf2: GeoDataFrame对象或None，如为None则从gdf2_path读取
    gdf1_path: str, 第一个shapefile的路径
    gdf2_path: str, 第二个shapefile的路径
    keep_columns1: list, 需要保留的gdf1字段列表，默认保留所有字段
    keep_columns2: list, 需要保留的gdf2字段列表，默认保留所有字段
    
    返回:
    GeoDataFrame: 相交后的结果
    """
    # 如果传入的是路径，则读取数据
    if gdf1 is None and gdf1_path is not None:
        gdf1 = gpd.read_file(gdf1_path, encoding='utf-8')
    if gdf2 is None and gdf2_path is not None:
        gdf2 = gpd.read_file(gdf2_path, encoding='utf-8')
        
    # 确保两个GeoDataFrame使用相同的坐标系
    if gdf1.crs != gdf2.crs:
        gdf2 = gdf2.to_crs(gdf1.crs)
    
    # 筛选需要保留的列
    if keep_columns1 is not None:
        keep_columns1 = [col for col in keep_columns1 if col in gdf1.columns]
        keep_columns1.append(gdf1.geometry.name)
        gdf1 = gdf1[keep_columns1]
    
    if keep_columns2 is not None:
        keep_columns2 = [col for col in keep_columns2 if col in gdf2.columns]
        keep_columns2.append(gdf2.geometry.name)
        gdf2 = gdf2[keep_columns2]
    
    # 执行空间相交
    intersect_gdf = gpd.overlay(gdf1, gdf2, how='intersection')
    
    # 计算相交后的面积
    intersect_gdf['Intersect_area'] = intersect_gdf.area
    
    return intersect_gdf
# 筛选耕地并保存
def crop_filter(soil_type_path,field_name,value_list, crop_path):
    # 读取土类图
    soil_type_gdf = gpd.read_file(soil_type_path,encoding='utf-8')
    # 筛选耕地
    crop_gdf = soil_type_gdf[soil_type_gdf[field_name].isin(value_list)]
    # 计算一个投影面积
    crop_gdf['P_area'] = crop_gdf.area
    # 保存
    crop_gdf.to_file(crop_path, driver='ESRI Shapefile',encoding='utf-8')

In [3]:
# 保存路径
save_dir = r'G:\soil_property_result\qzs\grade_evaluation'

In [4]:
# 土类图路径
soil_type_path = r"F:\cache_data\shp_file\qz\ele_qz\qz_soiltype_RandomForestEntr_20250227.shp"

In [ ]:
# 筛选耕地并保存
crop_path = os.path.join(save_dir, "shp", "crop.shp")
select_field_name = 'DLMC'
selected_value_list = ['水田','旱地','水浇地']
crop_filter(soil_type_path, select_field_name,selected_value_list,crop_path)

In [11]:
# 读取耕地
crop_gdf = gpd.read_file(crop_path,encoding='utf-8')

In [ ]:
crop_gdf.columns

In [14]:
# 按照唯一字段列表进行融合
def unique_field_merge(gdf, unique_field_list):
    # 按照唯一字段列表进行融合
    gdf = gdf.dissolve(by=unique_field_list)
    # 单部件
    gdf = gdf.explode(index_parts=False)
    return gdf

# 按照唯一字段列表进行融合
dissolve_field_list = ['BSM','DLMC','DLBM', 'DLMC','ZLDWDM','ZLDWMC','Class1','Class1_ts','Class1_yl','Class1_tl']
# save_path 
dissolve_path = os.path.join(save_dir, "shp", "crop_dissolve.shp")
crops_gdf = unique_field_merge(crop_gdf, dissolve_field_list)

In [ ]:
crops_gdf['BSM'].unique()

In [9]:
# 保存
crop_gdf.to_file(dissolve_path, driver='ESRI Shapefile',encoding='utf-8')